# Cost of Plutocracy

The aim of this article is to analyze some of the biggest DAOs in the crypto space and see how many of them are influenced by token holders which own a disproportionate amount of tokens compared to most voters.

In [46]:
# sets up the pynb environment
import os
import sys

import pandas as pd
import numpy as np

from libs.data_processing.filters import get_quartile_by_vp


module_path = os.path.abspath(os.path.join("."))
if module_path not in sys.path:
    sys.path.append(module_path)

The data used for this article came from data sources that specialize in providing data on DAO governance, such as DeepDAO and Snapshot. Coingecko was also used to find market data for DAO native tokens.

Using this data, two spreadsheets were compiled which act as the local database for this analysis. Each spreadsheet contains each voter's choice and voting power for the last five proposals each DAO opened. **One spreadsheet filters out "whales"** which, in the context of this analysis, are voters of each proposal which have voting power **at or above the 95th percentile of voting power for that proposal**.

In [51]:
plutocracy_report_data = pd.read_excel(
    "./plutocracy_report.xls", sheet_name=None, engine="openpyxl"
)
plutocracy_report_data_filtered = pd.read_excel(
    "./plutocracy_report_filtered.xls", sheet_name=None, engine="openpyxl"
)

With this, we can begin asking ourselves some questions. Such as: **How many whales have voted in the last five proposals for each of the DAOs we analyzed?**

In [65]:
def get_number_of_whales_to_all_voters_ratio(
    dao_proposals: dict[str, pd.DataFrame],
    dao_proposals_filtered: dict[str, pd.DataFrame],
) -> list[dict[str, int]]:
    ratios = []
    for organization in dao_proposals.keys():
        all_proposals = dao_proposals[organization]
        all_proposals_filtered = dao_proposals_filtered[organization]
        tally = {organization: [0, 0]}

        tally[organization][0] += all_proposals.shape[0] - all_proposals_filtered.shape[0]
        tally[organization][1] = all_proposals.shape[0]
        ratios.append(tally)

    return ratios


voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [66]:
pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    index=[list(result.items())[0][0] for result in voting_ratios],
    columns=["# of whales", "all voters"],
)


,# of whales,all voters
Uniswap,4,80
ENS,5,100
Wonderland,5,100
Lido,5,100
Frax,5,93
OlympusDAO,5,100
Decentraland,5,93
Curve Finance,4,73
Radicle,5,64
Fei,5,100
